In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [2]:
df_patient = pd.read_csv("./smuai/train_data_clinical_patient.csv")
df_rna = pd.read_csv("./smuai/train_data_rna-seq.csv")
print("train_data_clinical_patient shape > ",df_patient.shape)
print("train_data_rna-seq shape > ",df_rna.shape)

train_data_clinical_patient shape >  (2697, 37)
train_data_rna-seq shape >  (20531, 2765)


In [3]:
df_rna = df_rna.transpose()
rna_indexs = df_rna.index
new_rna_index = []
for rna_index in rna_indexs:
    if rna_index == 'Hugo_Symbol' or rna_index == 'Entrez_Gene_Id':
        new_rna_index.append(rna_index)
        continue
    r_index = rna_index.split('-')
    new_rna_index.append('{}-{}-{}'.format(r_index[0],r_index[1],r_index[2]))
    
print(new_rna_index[0])
df_rna.index = new_rna_index

df_rna = df_rna.transpose()
df_rna

Hugo_Symbol


,Hugo_Symbol,Entrez_Gene_Id,TCGA-3M-AB46,TCGA-3M-AB47,TCGA-B7-5816,TCGA-B7-5818,TCGA-B7-A5TI,TCGA-B7-A5TJ,TCGA-B7-A5TK,TCGA-B7-A5TN,...,TCGA-VW-A8FI,TCGA-W9-A837,TCGA-WH-A86K,TCGA-WY-A858,TCGA-WY-A859,TCGA-WY-A85A,TCGA-WY-A85B,TCGA-WY-A85C,TCGA-WY-A85D,TCGA-WY-A85E
0,NaN,100130426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-6.6197,-6.6197,-6.6197,-6.6197,-6.6197,-6.6197,-6.6197,-6.6197,-6.6197,-6.6197
1,NaN,100133144,-0.6912,-0.4056,-2.0249,-1.7479,-0.672,0.2729,-1.6346,-0.0469,...,1.1529,0.5938,0.4161,-0.0182,-1.0264,-0.4623,0.0507,0.4778,-0.4763,0.2876
2,NaN,100134869,-0.0569,-1.1995,-0.5616,1.2902,-0.0517,0.1636,-3.7799,0.0582,...,0.7471,1.0861,0.044,0.2981,-0.4821,0.2481,0.5676,0.4846,-0.6968,-0.9707
3,NaN,10357,0.7839,-0.0397,-2.7656,-1.0076,0.1231,1.2847,0.6795,0.6711,...,-0.3517,-0.8943,1.0942,0.7643,-0.4045,-0.4197,-0.2125,-0.5183,-0.8822,-0.009
4,NaN,10431,2.6285,0.1811,-0.254,-0.2604,1.3377,-0.1858,1.2572,0.4134,...,-0.381,-0.1261,-0.337,0.758,1.2928,0.2058,0.2314,-0.6588,0.4685,0.6967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20526,ZYG11A,440590,0.6274,-0.5807,-1.4317,-1.4317,-0.873,-0.732,-0.7461,-0.9315,...,0.7317,-1.2143,-1.2143,-0.466,0.11,-1.2143,-0.2231,0.9422,-1.2143,1.1519
20527,ZYG11B,79699,0.5821,0.3698,-0.8969,-1.6536,-0.9848,-0.9928,0.7091,-1.2326,...,-1.363,0.1147,0.0871,-0.9503,0.8521,0.4425,0.3485,1.0692,-0.1277,0.7383
20528,ZYX,7791,-0.8533,-0.1465,-0.0919,0.3913,-0.5291,0.1615,0.8531,0.4881,...,2.367,-1.0349,-0.184,0.3662,-0.442,-0.9402,-0.8402,-0.4389,-0.6813,-0.3227
20529,ZZEF1,23140,-1.6618,-0.0619,-0.3589,-1.5988,-0.6019,-0.1565,-1.6103,-1.1189,...,-1.733,0.2662,-0.5119,-0.9516,-0.6513,-0.1893,0.1042,0.7413,-0.5001,-0.4875


In [4]:
df_rna = df_rna.dropna(axis=0)

In [5]:
df_rna

,Hugo_Symbol,Entrez_Gene_Id,TCGA-3M-AB46,TCGA-3M-AB47,TCGA-B7-5816,TCGA-B7-5818,TCGA-B7-A5TI,TCGA-B7-A5TJ,TCGA-B7-A5TK,TCGA-B7-A5TN,...,TCGA-VW-A8FI,TCGA-W9-A837,TCGA-WH-A86K,TCGA-WY-A858,TCGA-WY-A859,TCGA-WY-A85A,TCGA-WY-A85B,TCGA-WY-A85C,TCGA-WY-A85D,TCGA-WY-A85E
29,A1BG,1,0.2076,0.4276,-0.9922,-1.0179,-0.4527,-0.5642,0.5899,-0.1583,...,0.8868,-1.4281,2.0103,1.366,-0.155,0.0365,0.1544,0.6591,1.8078,0.6973
32,A2LD1,87769,1.2071,0.6262,0.3858,-2.6991,0.8135,2.8214,-1.3555,0.387,...,1.4632,-0.3344,0.2268,0.368,-0.4358,-0.0906,0.0154,-0.3613,-0.9304,-1.1419
33,A2M,2,-0.7818,0.9275,-0.4149,-1.2313,0.4039,-0.7086,1.1713,0.4539,...,-0.6719,-0.9471,0.2347,2.3168,-1.4888,-0.3067,0.85,-0.0253,0.0875,-1.1735
34,A2ML1,144568,-0.8253,-0.6354,-0.4633,0.878,0.6896,-0.7604,-0.8253,0.9142,...,-0.3687,0.7322,1.3826,-0.7388,0.6714,0.9726,-0.8925,-0.9871,0.8196,0.4283
35,A4GALT,53947,-0.9673,0.4636,1.1193,-0.5319,-0.0799,-0.5281,-0.107,-0.1437,...,-0.7263,-0.6376,1.4728,1.7026,-1.4383,-0.7398,-1.0079,0.1267,-0.3884,-1.5843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20526,ZYG11A,440590,0.6274,-0.5807,-1.4317,-1.4317,-0.873,-0.732,-0.7461,-0.9315,...,0.7317,-1.2143,-1.2143,-0.466,0.11,-1.2143,-0.2231,0.9422,-1.2143,1.1519
20527,ZYG11B,79699,0.5821,0.3698,-0.8969,-1.6536,-0.9848,-0.9928,0.7091,-1.2326,...,-1.363,0.1147,0.0871,-0.9503,0.8521,0.4425,0.3485,1.0692,-0.1277,0.7383
20528,ZYX,7791,-0.8533,-0.1465,-0.0919,0.3913,-0.5291,0.1615,0.8531,0.4881,...,2.367,-1.0349,-0.184,0.3662,-0.442,-0.9402,-0.8402,-0.4389,-0.6813,-0.3227
20529,ZZEF1,23140,-1.6618,-0.0619,-0.3589,-1.5988,-0.6019,-0.1565,-1.6103,-1.1189,...,-1.733,0.2662,-0.5119,-0.9516,-0.6513,-0.1893,0.1042,0.7413,-0.5001,-0.4875


In [6]:
list_df_rna = df_rna['Hugo_Symbol']
list_df_rna = list_df_rna.dropna(axis=0)

del df_rna['Hugo_Symbol']
del df_rna['Entrez_Gene_Id']

In [7]:
df_rna.index = list_df_rna
df_rna

,TCGA-3M-AB46,TCGA-3M-AB47,TCGA-B7-5816,TCGA-B7-5818,TCGA-B7-A5TI,TCGA-B7-A5TJ,TCGA-B7-A5TK,TCGA-B7-A5TN,TCGA-BR-4187,TCGA-BR-4280,...,TCGA-VW-A8FI,TCGA-W9-A837,TCGA-WH-A86K,TCGA-WY-A858,TCGA-WY-A859,TCGA-WY-A85A,TCGA-WY-A85B,TCGA-WY-A85C,TCGA-WY-A85D,TCGA-WY-A85E
Hugo_Symbol,,,,,,,,,,,,,,,,,,,,,
A1BG,0.2076,0.4276,-0.9922,-1.0179,-0.4527,-0.5642,0.5899,-0.1583,-1.068,-1.073,...,0.8868,-1.4281,2.0103,1.366,-0.155,0.0365,0.1544,0.6591,1.8078,0.6973
A2LD1,1.2071,0.6262,0.3858,-2.6991,0.8135,2.8214,-1.3555,0.387,0.0664,1.2767,...,1.4632,-0.3344,0.2268,0.368,-0.4358,-0.0906,0.0154,-0.3613,-0.9304,-1.1419
A2M,-0.7818,0.9275,-0.4149,-1.2313,0.4039,-0.7086,1.1713,0.4539,1.618,-1.1378,...,-0.6719,-0.9471,0.2347,2.3168,-1.4888,-0.3067,0.85,-0.0253,0.0875,-1.1735
A2ML1,-0.8253,-0.6354,-0.4633,0.878,0.6896,-0.7604,-0.8253,0.9142,-0.0017,-0.8253,...,-0.3687,0.7322,1.3826,-0.7388,0.6714,0.9726,-0.8925,-0.9871,0.8196,0.4283
A4GALT,-0.9673,0.4636,1.1193,-0.5319,-0.0799,-0.5281,-0.107,-0.1437,0.6296,-1.7409,...,-0.7263,-0.6376,1.4728,1.7026,-1.4383,-0.7398,-1.0079,0.1267,-0.3884,-1.5843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11A,0.6274,-0.5807,-1.4317,-1.4317,-0.873,-0.732,-0.7461,-0.9315,-0.5846,-1.2122,...,0.7317,-1.2143,-1.2143,-0.466,0.11,-1.2143,-0.2231,0.9422,-1.2143,1.1519
ZYG11B,0.5821,0.3698,-0.8969,-1.6536,-0.9848,-0.9928,0.7091,-1.2326,1.4348,-0.9524,...,-1.363,0.1147,0.0871,-0.9503,0.8521,0.4425,0.3485,1.0692,-0.1277,0.7383
ZYX,-0.8533,-0.1465,-0.0919,0.3913,-0.5291,0.1615,0.8531,0.4881,0.3664,-0.7196,...,2.367,-1.0349,-0.184,0.3662,-0.442,-0.9402,-0.8402,-0.4389,-0.6813,-0.3227


In [8]:
df_rna = df_rna.transpose()
df_rna['PATIENT_ID'] = df_rna.index

In [9]:
df_rna

Hugo_Symbol,A1BG,A2LD1,A2M,A2ML1,A4GALT,AAAS,AACS,AADAT,AAGAB,AAK1,...,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,PATIENT_ID
TCGA-3M-AB46,0.2076,1.2071,-0.7818,-0.8253,-0.9673,0.0646,0.188,0.7385,-0.6939,-1.2098,...,2.6983,-1.0061,-0.4243,-1.1151,0.6274,0.5821,-0.8533,-1.6618,0.2748,TCGA-3M-AB46
TCGA-3M-AB47,0.4276,0.6262,0.9275,-0.6354,0.4636,-0.5795,-1.2742,-1.4299,-0.4931,0.0009,...,-0.5029,-1.1278,0.6149,-0.7507,-0.5807,0.3698,-0.1465,-0.0619,0.5492,TCGA-3M-AB47
TCGA-B7-5816,-0.9922,0.3858,-0.4149,-0.4633,1.1193,-0.6825,-0.7953,0.1968,0.0606,-0.664,...,0.1416,-0.0425,-0.9111,-1.1933,-1.4317,-0.8969,-0.0919,-0.3589,-1.4745,TCGA-B7-5816
TCGA-B7-5818,-1.0179,-2.6991,-1.2313,0.878,-0.5319,-0.0726,1.1271,-0.2101,-0.0258,-1.0637,...,0.7458,-2.5996,-5.5211,1.357,-1.4317,-1.6536,0.3913,-1.5988,-2.7521,TCGA-B7-5818
TCGA-B7-A5TI,-0.4527,0.8135,0.4039,0.6896,-0.0799,0.0263,2.1708,-1.3087,1.1859,-1.087,...,1.0328,-1.617,-1.072,-0.4373,-0.873,-0.9848,-0.5291,-0.6019,0.0214,TCGA-B7-A5TI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-WY-A85A,0.0365,-0.0906,-0.3067,0.9726,-0.7398,0.8323,0.0021,1.4102,-0.4497,-0.3156,...,-0.4914,-0.3775,-0.6975,0.0466,-1.2143,0.4425,-0.9402,-0.1893,0.4236,TCGA-WY-A85A
TCGA-WY-A85B,0.1544,0.0154,0.85,-0.8925,-1.0079,-0.0945,-0.87,1.7563,0.4346,-0.4302,...,1.1282,-0.1515,0.5935,0.3402,-0.2231,0.3485,-0.8402,0.1042,1.0122,TCGA-WY-A85B
TCGA-WY-A85C,0.6591,-0.3613,-0.0253,-0.9871,0.1267,0.8454,-0.4925,1.8238,-0.4032,-0.0873,...,-0.7663,0.3284,0.7869,0.9922,0.9422,1.0692,-0.4389,0.7413,1.2262,TCGA-WY-A85C
TCGA-WY-A85D,1.8078,-0.9304,0.0875,0.8196,-0.3884,0.9347,0.2701,0.4882,-1.4738,-0.7057,...,-0.3125,-1.2386,-1.1186,0.6909,-1.2143,-0.1277,-0.6813,-0.5001,0.8035,TCGA-WY-A85D


In [10]:
dd = pd.DataFrame({'PATIENT_ID':df_patient['PATIENT_ID'],
                  'CANCER_TYPE_ACRONYM':df_patient['CANCER_TYPE_ACRONYM']})

In [11]:
#df_concat = pd.concat([df_rna, df_cna])

df = pd.merge(df_rna, dd, on='PATIENT_ID')

df.index = df['PATIENT_ID']
df

,A1BG,A2LD1,A2M,A2ML1,A4GALT,AAAS,AACS,AADAT,AAGAB,AAK1,...,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,PATIENT_ID,CANCER_TYPE_ACRONYM
PATIENT_ID,,,,,,,,,,,,,,,,,,,,,
TCGA-3M-AB46,0.2076,1.2071,-0.7818,-0.8253,-0.9673,0.0646,0.188,0.7385,-0.6939,-1.2098,...,-1.0061,-0.4243,-1.1151,0.6274,0.5821,-0.8533,-1.6618,0.2748,TCGA-3M-AB46,STAD
TCGA-3M-AB47,0.4276,0.6262,0.9275,-0.6354,0.4636,-0.5795,-1.2742,-1.4299,-0.4931,0.0009,...,-1.1278,0.6149,-0.7507,-0.5807,0.3698,-0.1465,-0.0619,0.5492,TCGA-3M-AB47,STAD
TCGA-B7-5816,-0.9922,0.3858,-0.4149,-0.4633,1.1193,-0.6825,-0.7953,0.1968,0.0606,-0.664,...,-0.0425,-0.9111,-1.1933,-1.4317,-0.8969,-0.0919,-0.3589,-1.4745,TCGA-B7-5816,STAD
TCGA-B7-5818,-1.0179,-2.6991,-1.2313,0.878,-0.5319,-0.0726,1.1271,-0.2101,-0.0258,-1.0637,...,-2.5996,-5.5211,1.357,-1.4317,-1.6536,0.3913,-1.5988,-2.7521,TCGA-B7-5818,STAD
TCGA-B7-A5TI,-0.4527,0.8135,0.4039,0.6896,-0.0799,0.0263,2.1708,-1.3087,1.1859,-1.087,...,-1.617,-1.072,-0.4373,-0.873,-0.9848,-0.5291,-0.6019,0.0214,TCGA-B7-A5TI,STAD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-WY-A85A,0.0365,-0.0906,-0.3067,0.9726,-0.7398,0.8323,0.0021,1.4102,-0.4497,-0.3156,...,-0.3775,-0.6975,0.0466,-1.2143,0.4425,-0.9402,-0.1893,0.4236,TCGA-WY-A85A,LGG
TCGA-WY-A85B,0.1544,0.0154,0.85,-0.8925,-1.0079,-0.0945,-0.87,1.7563,0.4346,-0.4302,...,-0.1515,0.5935,0.3402,-0.2231,0.3485,-0.8402,0.1042,1.0122,TCGA-WY-A85B,LGG
TCGA-WY-A85C,0.6591,-0.3613,-0.0253,-0.9871,0.1267,0.8454,-0.4925,1.8238,-0.4032,-0.0873,...,0.3284,0.7869,0.9922,0.9422,1.0692,-0.4389,0.7413,1.2262,TCGA-WY-A85C,LGG


In [12]:
Y = df['CANCER_TYPE_ACRONYM']
del df['CANCER_TYPE_ACRONYM']
del df['PATIENT_ID']

In [13]:
#data split 
X_train, X_test, y_train, y_test = train_test_split(df, Y, test_size=0.3, random_state=10)

model = RandomForestClassifier(criterion='entropy', n_jobs=2, random_state=1)
model.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_jobs=2, random_state=1)

In [14]:
model.predict(X_test)

array(['LIHC', 'COAD', 'STAD', 'STAD', 'LGG', 'SKCM', 'SKCM', 'COAD',
       'LIHC', 'LUAD', 'COAD', 'SKCM', 'SKCM', 'LIHC', 'LIHC', 'STAD',
       'COAD', 'SKCM', 'LGG', 'SKCM', 'STAD', 'LGG', 'COAD', 'COAD',
       'STAD', 'LGG', 'LGG', 'LUAD', 'LGG', 'LGG', 'STAD', 'SKCM', 'COAD',
       'STAD', 'LGG', 'STAD', 'LIHC', 'SKCM', 'LIHC', 'LGG', 'SKCM',
       'LIHC', 'LGG', 'SKCM', 'SKCM', 'LUAD', 'STAD', 'LGG', 'COAD',
       'LGG', 'SKCM', 'LUAD', 'LGG', 'SKCM', 'LGG', 'COAD', 'STAD', 'LGG',
       'LIHC', 'LUAD', 'LGG', 'STAD', 'LUAD', 'STAD', 'LUAD', 'COAD',
       'COAD', 'LGG', 'LIHC', 'SKCM', 'STAD', 'COAD', 'LUAD', 'LUAD',
       'LGG', 'COAD', 'STAD', 'COAD', 'LIHC', 'LIHC', 'LGG', 'LGG',
       'SKCM', 'LUAD', 'LGG', 'COAD', 'LIHC', 'LUAD', 'LGG', 'COAD',
       'LGG', 'LUAD', 'LUAD', 'COAD', 'LUAD', 'STAD', 'LGG', 'LGG',
       'SKCM', 'LGG', 'COAD', 'LGG', 'LUAD', 'LIHC', 'LGG', 'SKCM',
       'STAD', 'LIHC', 'STAD', 'SKCM', 'SKCM', 'LGG', 'LGG', 'LGG',
       'STAD', 'LGG',

In [15]:
model.score(X_test, y_test)

0.9987228607918263

In [16]:
print("훈련 세트 정확도: {:.3f}".format(model.score(X_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(model.score(X_test, y_test)))

훈련 세트 정확도: 1.000
테스트 세트 정확도: 0.999


In [17]:
df_test_patient = pd.read_csv("./smuai/test_data_clinical_patient_v1.csv")
df_test_rna = pd.read_csv("./smuai/test_data_rna-seq_v1.csv")

In [18]:
df_test_patient

,id,AGE,SEX,AJCC_PATHOLOGIC_TUMOR_STAGE,AJCC_STAGING_EDITION,DAYS_LAST_FOLLOWUP,DAYS_TO_BIRTH,DAYS_TO_INITIAL_PATHOLOGIC_DIAGNOSIS,ETHNICITY,FORM_COMPLETION_DATE,...,WEIGHT,CENTER,IN_PANCANPATHWAYS_FREEZE,OS_STATUS,OS_MONTHS,DSS_STATUS,DSS_MONTHS,DFS_STATUS,DFS_MONTHS,PFS_STATUS
0,patient_1,76,Female,STAGE IIIA,6TH,0.0,-27893,0,Not Hispanic Or Latino,06/06/2011,...,NaN,Asterand,Yes,1:DECEASED,4.076668,NaN,4.076668,NaN,NaN,0:CENSORED
1,patient_2,72,Male,STAGE IA,7TH,830.0,-26317,0,Not Hispanic Or Latino,6/20/12,...,NaN,Washington University - Emory,Yes,0:LIVING,27.287372,0:ALIVE OR DEAD TUMOR FREE,27.287372,1:Recurred/Progressed,7.495808,1:PROGRESSION
2,patient_3,24,Male,STAGE II,7TH,601.0,-8951,0,Not Hispanic Or Latino,8/15/14,...,62.0,UCSF,Yes,0:LIVING,19.758688,0:ALIVE OR DEAD TUMOR FREE,19.758688,1:Recurred/Progressed,1.578065,1:PROGRESSION
3,patient_4,59,Male,STAGE IIIA,7TH,500.0,-21733,0,Not Hispanic Or Latino,8/15/14,...,78.0,UCSF,Yes,0:LIVING,16.438176,0:ALIVE OR DEAD TUMOR FREE,16.438176,1:Recurred/Progressed,11.802610,1:PROGRESSION
4,patient_5,65,Male,STAGE IIIA,7TH,564.0,-23776,0,Not Hispanic Or Latino,5/13/11,...,NaN,Greater Poland Cancer Center,Yes,0:LIVING,18.542263,0:ALIVE OR DEAD TUMOR FREE,18.542263,NaN,NaN,1:PROGRESSION
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,patient_70,63,Female,STAGE IIIC,7TH,571.0,-23245,0,Not Hispanic Or Latino,12/18/13,...,67.0,University of North Carolina,Yes,0:LIVING,18.772397,0:ALIVE OR DEAD TUMOR FREE,18.772397,NaN,NaN,1:PROGRESSION
70,patient_71,71,Male,STAGE IIIC,7TH,384.0,-26118,0,Not Hispanic Or Latino,8/19/13,...,104.0,Roswell,Yes,1:DECEASED,22.125785,1:DEAD WITH TUMOR,22.125785,NaN,NaN,1:PROGRESSION
71,patient_72,41,Male,NaN,NaN,588.0,-15092,0,Not Hispanic Or Latino,08/02/2013,...,NaN,Fondazione-Besta,Yes,0:LIVING,19.331295,0:ALIVE OR DEAD TUMOR FREE,19.331295,NaN,NaN,1:PROGRESSION
72,patient_73,68,Female,STAGE IIC,7TH,NaN,-25000,0,Not Hispanic Or Latino,04/01/2014,...,89.0,Roswell,Yes,1:DECEASED,21.369629,1:DEAD WITH TUMOR,21.369629,NaN,NaN,1:PROGRESSION


In [19]:
df_test_rna

,Hugo_Symbol,Entrez_Gene_Id,patient_34,patient_35,patient_5,patient_65,patient_7,patient_8,patient_9,patient_1,...,patient_52,patient_53,patient_54,patient_28,patient_68,patient_57,patient_58,patient_59,patient_72,patient_61
0,NaN,100130426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-6.6197,-6.6197,-6.6197,-6.6197,-6.6197,-6.6197,-6.6197,-6.6197,-6.6197,-6.6197
1,NaN,100133144,0.0313,-1.1888,-0.6275,-2.8225,-0.9355,0.2755,-0.5159,-3.6531,...,-0.0104,-0.4469,-0.2674,-1.4728,-1.1543,-0.0359,-3.1094,-0.7935,-3.1094,-1.4103
2,NaN,100134869,-1.9899,0.0618,-0.6568,-2.0187,-0.2693,1.4479,1.0652,-1.2866,...,-0.6447,-0.7510,-0.4568,-0.9079,-0.1149,-0.9998,-0.6871,0.8822,-0.4106,0.4940
3,NaN,10357,0.8538,-1.4139,-1.6290,-1.6207,-2.0586,-1.8798,-0.6720,-0.8844,...,0.2089,1.8635,1.5797,0.0796,0.3014,0.0396,-1.1425,-0.5699,-0.2267,0.2357
4,NaN,10431,0.0726,-1.4780,-1.4820,0.1970,-1.0236,0.1647,0.5723,1.0053,...,0.3580,0.8525,0.2391,0.5402,0.6718,0.2211,2.4127,0.6833,-0.5026,1.6788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20526,ZYG11A,440590,-1.4317,1.1030,-0.6893,-0.3313,-1.0208,-1.4317,-0.2368,1.8149,...,0.6812,-0.3444,-1.2143,-1.2143,-0.1383,-0.5836,-0.0645,-0.6352,0.5606,-1.2143
20527,ZYG11B,79699,-0.4268,-1.1815,-0.9811,-1.2863,-0.3284,-1.4403,-1.2965,0.7038,...,-0.0309,1.1275,0.8424,0.6774,1.8079,-1.3332,-3.6953,-0.6294,-0.7373,-2.0814
20528,ZYX,7791,-0.0860,0.4660,0.7733,1.8049,1.3232,0.5540,-0.2708,1.1159,...,-1.1755,-0.5240,-1.6614,-0.2644,-0.6075,0.5624,1.3264,-0.5177,-0.3267,0.6660
20529,ZZEF1,23140,-0.2940,-1.3591,-1.4425,0.3220,0.5374,-2.5013,-0.5578,-0.3484,...,-0.8474,-0.8440,0.0883,0.2331,-2.0918,-1.0157,-0.9135,-0.4802,-1.3742,-1.6413


In [20]:
df_test_rna = df_test_rna.dropna()
list_df_rna_test = df_test_rna['Hugo_Symbol']
df_test_rna.index = list_df_rna_test
df_test_rna

,Hugo_Symbol,Entrez_Gene_Id,patient_34,patient_35,patient_5,patient_65,patient_7,patient_8,patient_9,patient_1,...,patient_52,patient_53,patient_54,patient_28,patient_68,patient_57,patient_58,patient_59,patient_72,patient_61
Hugo_Symbol,,,,,,,,,,,,,,,,,,,,,
A1BG,A1BG,1,-0.7970,-1.4861,0.1948,1.3791,0.8012,-1.3639,-1.2248,0.3409,...,0.2190,-2.0785,-0.4279,-1.4864,-0.4417,0.9471,-0.7556,0.1781,0.9657,0.3516
A2LD1,A2LD1,87769,-0.4542,-0.2880,-0.2676,0.4978,0.5735,-0.5469,0.3127,-0.4666,...,-0.3464,-0.0686,0.1903,-0.1848,-0.1319,0.9772,-0.5544,-0.8007,1.8816,1.4004
A2M,A2M,2,0.4107,0.4235,0.8486,1.6071,1.6144,0.1215,-0.5271,1.5034,...,0.8221,0.7054,0.0560,1.7167,-0.9979,0.2529,1.2668,-0.9036,0.1425,-0.4349
A2ML1,A2ML1,144568,0.4798,-0.8253,-0.7489,-0.8253,0.8943,2.2609,-0.7708,-0.8245,...,0.0069,-0.7039,1.5321,-1.5779,0.7415,1.5292,-1.4734,0.5082,1.9203,0.5836
A4GALT,A4GALT,53947,-0.4802,1.1270,0.8944,1.9917,1.2996,1.0469,0.1620,1.3905,...,-0.6327,0.4112,-1.6394,1.3489,-0.9535,0.0796,2.3380,-0.3016,0.2481,0.8949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11A,ZYG11A,440590,-1.4317,1.1030,-0.6893,-0.3313,-1.0208,-1.4317,-0.2368,1.8149,...,0.6812,-0.3444,-1.2143,-1.2143,-0.1383,-0.5836,-0.0645,-0.6352,0.5606,-1.2143
ZYG11B,ZYG11B,79699,-0.4268,-1.1815,-0.9811,-1.2863,-0.3284,-1.4403,-1.2965,0.7038,...,-0.0309,1.1275,0.8424,0.6774,1.8079,-1.3332,-3.6953,-0.6294,-0.7373,-2.0814
ZYX,ZYX,7791,-0.0860,0.4660,0.7733,1.8049,1.3232,0.5540,-0.2708,1.1159,...,-1.1755,-0.5240,-1.6614,-0.2644,-0.6075,0.5624,1.3264,-0.5177,-0.3267,0.6660


In [21]:
del df_test_rna['Hugo_Symbol']
del df_test_rna['Entrez_Gene_Id']
df_test_rna

,patient_34,patient_35,patient_5,patient_65,patient_7,patient_8,patient_9,patient_1,patient_18,patient_10,...,patient_52,patient_53,patient_54,patient_28,patient_68,patient_57,patient_58,patient_59,patient_72,patient_61
Hugo_Symbol,,,,,,,,,,,,,,,,,,,,,
A1BG,-0.7970,-1.4861,0.1948,1.3791,0.8012,-1.3639,-1.2248,0.3409,-0.5841,-0.3315,...,0.2190,-2.0785,-0.4279,-1.4864,-0.4417,0.9471,-0.7556,0.1781,0.9657,0.3516
A2LD1,-0.4542,-0.2880,-0.2676,0.4978,0.5735,-0.5469,0.3127,-0.4666,0.1704,-0.4939,...,-0.3464,-0.0686,0.1903,-0.1848,-0.1319,0.9772,-0.5544,-0.8007,1.8816,1.4004
A2M,0.4107,0.4235,0.8486,1.6071,1.6144,0.1215,-0.5271,1.5034,0.7686,-0.5841,...,0.8221,0.7054,0.0560,1.7167,-0.9979,0.2529,1.2668,-0.9036,0.1425,-0.4349
A2ML1,0.4798,-0.8253,-0.7489,-0.8253,0.8943,2.2609,-0.7708,-0.8245,-0.6400,-0.6581,...,0.0069,-0.7039,1.5321,-1.5779,0.7415,1.5292,-1.4734,0.5082,1.9203,0.5836
A4GALT,-0.4802,1.1270,0.8944,1.9917,1.2996,1.0469,0.1620,1.3905,0.6822,0.2934,...,-0.6327,0.4112,-1.6394,1.3489,-0.9535,0.0796,2.3380,-0.3016,0.2481,0.8949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11A,-1.4317,1.1030,-0.6893,-0.3313,-1.0208,-1.4317,-0.2368,1.8149,-0.2688,0.9078,...,0.6812,-0.3444,-1.2143,-1.2143,-0.1383,-0.5836,-0.0645,-0.6352,0.5606,-1.2143
ZYG11B,-0.4268,-1.1815,-0.9811,-1.2863,-0.3284,-1.4403,-1.2965,0.7038,0.4275,0.2750,...,-0.0309,1.1275,0.8424,0.6774,1.8079,-1.3332,-3.6953,-0.6294,-0.7373,-2.0814
ZYX,-0.0860,0.4660,0.7733,1.8049,1.3232,0.5540,-0.2708,1.1159,1.1149,-0.8993,...,-1.1755,-0.5240,-1.6614,-0.2644,-0.6075,0.5624,1.3264,-0.5177,-0.3267,0.6660


In [22]:
df_test_rna = df_test_rna.transpose()

In [23]:
df_test_rna

Hugo_Symbol,A1BG,A2LD1,A2M,A2ML1,A4GALT,AAAS,AACS,AADAT,AAGAB,AAK1,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
patient_34,-0.7970,-0.4542,0.4107,0.4798,-0.4802,-0.6607,-0.8752,0.1704,0.7078,-0.1265,...,0.6668,0.1833,-0.2432,-0.3486,0.1757,-1.4317,-0.4268,-0.0860,-0.2940,-0.0350
patient_35,-1.4861,-0.2880,0.4235,-0.8253,1.1270,-0.2843,-0.8806,0.8085,0.5371,-1.3420,...,-1.4181,-1.4764,0.4286,-0.2136,-0.4854,1.1030,-1.1815,0.4660,-1.3591,-0.9253
patient_5,0.1948,-0.2676,0.8486,-0.7489,0.8944,-0.5063,1.3039,-0.2225,-0.3322,-0.0475,...,-1.1046,-0.9107,0.4796,0.2064,-0.6044,-0.6893,-0.9811,0.7733,-1.4425,-0.7750
patient_65,1.3791,0.4978,1.6071,-0.8253,1.9917,1.0861,-0.5759,-0.6259,-0.7887,0.0257,...,-2.2675,-0.4971,0.8220,-0.0229,-0.5947,-0.3313,-1.2863,1.8049,0.3220,-1.9503
patient_7,0.8012,0.5735,1.6144,0.8943,1.2996,-0.5246,0.0183,-0.8706,-0.3116,-0.2014,...,-0.8591,-0.4480,0.3806,-0.2777,-0.7029,-1.0208,-0.3284,1.3232,0.5374,-0.6608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
patient_57,0.9471,0.9772,0.2529,1.5292,0.0796,-0.8843,-0.3288,0.1925,1.0290,-0.0127,...,0.0831,-0.9160,-1.0609,-0.9301,0.3490,-0.5836,-1.3332,0.5624,-1.0157,0.6091
patient_58,-0.7556,-0.5544,1.2668,-1.4734,2.3380,2.0684,0.0515,0.0015,-0.4252,-1.3135,...,-0.4510,1.2286,-1.5066,-1.0752,-0.7429,-0.0645,-3.6953,1.3264,-0.9135,-4.3633
patient_59,0.1781,-0.8007,-0.9036,0.5082,-0.3016,0.6352,-0.6799,1.1298,-0.4679,-0.3762,...,-1.9418,-0.5863,-0.5323,0.0291,0.1092,-0.6352,-0.6294,-0.5177,-0.4802,-1.3504
patient_72,0.9657,1.8816,0.1425,1.9203,0.2481,-0.5267,-0.5532,0.1002,1.7107,-0.4084,...,-0.8721,-2.1572,-1.5062,-1.5347,-0.2171,0.5606,-0.7373,-0.3267,-1.3742,0.0142


In [24]:
result = model.predict(df_test_rna)
result

array(['STAD', 'STAD', 'STAD', 'STAD', 'STAD', 'STAD', 'STAD', 'STAD',
       'STAD', 'LUAD', 'LUAD', 'LUAD', 'LUAD', 'LUAD', 'LUAD', 'LUAD',
       'LUAD', 'LUAD', 'LUAD', 'LUAD', 'LUAD', 'LUAD', 'LUAD', 'LIHC',
       'LIHC', 'LIHC', 'LIHC', 'LIHC', 'LIHC', 'LIHC', 'LIHC', 'LIHC',
       'LIHC', 'LIHC', 'LIHC', 'LIHC', 'SKCM', 'SKCM', 'SKCM', 'SKCM',
       'SKCM', 'SKCM', 'SKCM', 'SKCM', 'SKCM', 'SKCM', 'SKCM', 'SKCM',
       'SKCM', 'COAD', 'COAD', 'COAD', 'COAD', 'COAD', 'COAD', 'COAD',
       'COAD', 'COAD', 'COAD', 'COAD', 'LGG', 'LGG', 'LGG', 'LGG', 'LGG',
       'LGG', 'LGG', 'LGG', 'LGG', 'LGG', 'LGG', 'LGG', 'LGG', 'LGG'],
      dtype=object)

In [25]:
result_df = pd.DataFrame({
    'id':df_test_rna.index,
    'expected':result
})
result_df

,id,expected
0,patient_34,STAD
1,patient_35,STAD
2,patient_5,STAD
3,patient_65,STAD
4,patient_7,STAD
...,...,...
69,patient_57,LGG
70,patient_58,LGG
71,patient_59,LGG
72,patient_72,LGG


In [26]:
#정렬
result_df['index'] = pd.to_numeric(result_df.id.str.split('_').str[1])
result_df = result_df.sort_values('index')
result_df.index = result_df['index']
del result_df['index']
print(result_df)

               id expected
index                     
1       patient_1     STAD
2       patient_2     LUAD
3       patient_3     LIHC
4       patient_4     LIHC
5       patient_5     STAD
...           ...      ...
70     patient_70     SKCM
71     patient_71     SKCM
72     patient_72      LGG
73     patient_73     SKCM
74     patient_74     SKCM

[74 rows x 2 columns]


In [27]:
result_df.to_csv('./submission.csv', index=False, columns=['id','expected'])